In [1]:
# import relevant libraries
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:,.2f}'.format)

# Importing dataset

In [2]:
# import dataset
df = pd.read_csv('Final_Merged_File.csv',index_col = 0)
df.head()

,ITEM_ID,SLS_USD,Qty,Site_Level_Price_Index_WS,Site_Level_Price_Index_STU,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SITE_CREATION_DATE,SIC_CODE_DESC,SHORT_VERTICAL,CITY,STATE,COUNTRY,POSTAL_CODE,SELLING_ORG,CUSTOMER_CLASS,SHIP_TO_SITE_NUMBER,CORPORATE_CODE,LAST_MODIFIED_DATE,TERRITORY_REGION,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,IB_CONFIRMED_DATE,IB_STATUS,IB_STATUS_INACTIVE_REASON,SFDC_ID,DUNS_NUMBER,CUSTOMER_TRX_ID,DW_INVOICE_ID,TRX_DATE,TRX_AMT_USD,TRX_COST_USD,INVOICE_CURRENCY_CODE,SALES_CHANNEL,INVOICE_TYPE,QUANTITY,SHIP_TO_SITE_ID,SHIP_TO_CUSTOMER_ID,BILL_TO_CUSTOMER_ID,INVOICE_NUM,ORDER_TYPE,ORDER_NUM,PRODUCT_GROUP,PRODUCT_FAMILY,PRODUCT_MODEL,UOM,PFV_FAMILY,PFV_MODEL_GROUP,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Tasks,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Install_Bases,Contract_Category,Num_of_Inactive_Install_Bases
0,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,8/15/2013,2033-Canned fruits and specialties,FRUIT & VEGETABLE,SALEM,OR,US,97301,USA,END USER,102036258,NaN,9/30/2019,NW,Industrial,S,Over Stocked / Timing,4/10/2017,Active,NaN,001C00000187X5zIAE,78842640,8731929,108745302,2/5/2016,207.72,19.13,USD,Online,INVOICE,1.00,609636,117841,117841,3206875,STANDARD DOMESTIC,"511,759,211.00",SUPPLIES,CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,nan,NaN,NaN,6.00,0.00,0.00,6.00,No Contract,0.00
1,598740,644.37,3.00,588.99,644.37,0.91,113032,578406,3/25/2013,3721-Aircraft,AERO/AUTO,CRESTVIEW,FL,US,32539,USA,END USER,102005330,NaN,10/1/2019,SE,Industrial,S,NaN,5/8/2020,Active,NaN,001C00000158vAsIAI,43202248,9305874,276000052,12/16/2016,214.79,15.85,USD,Esker,INVOICE,1.00,578406,113032,97674,3294491,STANDARD DOMESTIC,"511,868,043.00",SUPPLIES,CIJ,VALUE PACK,Each,CIJ,CIJ - LEGACY,21.00,5.00,16.00,Email,58.00,62.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",4.00,FSMA,0.00
2,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,8/15/2013,2033-Canned fruits and specialties,FRUIT & VEGETABLE,SALEM,OR,US,97301,USA,END USER,102036258,NaN,9/30/2019,NW,Industrial,S,Over Stocked / Timing,4/10/2017,Active,NaN,001C00000187X5zIAE,78842640,9093354,223068431,8/26/2016,207.72,19.13,USD,Online,INVOICE,1.00,609636,117841,117841,3263477,STANDARD DOMESTIC,"511,823,154.00",SUPPLIES,CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,nan,NaN,NaN,6.00,0.00,0.00,6.00,No Contract,0.00
3,598740,644.37,3.00,588.99,644.37,0.91,113032,578406,3/25/2013,3721-Aircraft,AERO/AUTO,CRESTVIEW,FL,US,32539,USA,END USER,102005330,NaN,10/1/2019,SE,Industrial,S,NaN,5/8/2020,Active,NaN,001C00000158vAsIAI,43202248,9195970,247802392,10/19/2016,429.58,31.69,USD,Esker,INVOICE,2.00,578406,113032,97674,3278903,STANDARD DOMESTIC,"511,849,315.00",SUPPLIES,CIJ,VALUE PACK,Each,CIJ,CIJ - LEGACY,21.00,5.00,16.00,Email,58.00,62.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",4.00,FSMA,0.00
4,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,8/15/2013,2033-Canned fruits and specialties,FRUIT & VEGETABLE,SALEM,OR,US,97301,USA,END USER,102036258,NaN,9/30/2019,NW,Industrial,S,Over Stocked / Timing,4/10/2017,Active,NaN,001C00000187X5zIAE,78842640,9079815,219738849,8/19/2016,623.16,57.39,USD,Online,INVOICE,3.00,609636,117841,117841,3261624,STANDARD DOMESTIC,"511,823,154.00",SUPPLIES,CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,nan,NaN,NaN,6.00,0.00,0.00,6.00,No Contract,0.00


In [3]:
df.shape

(628077, 65)

In [4]:
# check duplicates
df.drop_duplicates(inplace = True)
df.shape

(628077, 65)

In [5]:
# make a copy and analyze on the copied data
data = df.copy()

# Filtering

In [6]:
# keep positive transactions only
data = data[data['TRX_AMT_USD'] > 0]

# keep only US
data = data[data['COUNTRY'] == "US"]
data.shape

(544668, 65)

# Checking Each Variable

## Converting date variables to datetime type

In [8]:
# set date columns to datetime 
data['TRX_DATE'] = pd.to_datetime(data['TRX_DATE'])
data['SITE_CREATION_DATE'] = pd.to_datetime(data['SITE_CREATION_DATE'])
data['LAST_MODIFIED_DATE'] = pd.to_datetime(data['LAST_MODIFIED_DATE'])
data['IB_CONFIRMED_DATE'] = pd.to_datetime(data['IB_CONFIRMED_DATE'])

## SFDC_ID: drop column

In [10]:
data.drop(["SFDC_ID"],axis = 1, inplace = True)

In [11]:
data.shape

(544668, 64)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544668 entries, 0 to 628076
Data columns (total 64 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544668 non-null  int64         
 1   SLS_USD                         544668 non-null  float64       
 2   Qty                             544668 non-null  float64       
 3   Site_Level_Price_Index_WS       544668 non-null  float64       
 4   Site_Level_Price_Index_STU      544668 non-null  float64       
 5   Site_Level_Price_Index          544668 non-null  float64       
 6   CUSTOMER_ID                     544668 non-null  int64         
 7   CUSTOMER_SITE_ID                544668 non-null  int64         
 8   SITE_CREATION_DATE              544668 non-null  datetime64[ns]
 9   SIC_CODE_DESC                   544668 non-null  object        
 10  SHORT_VERTICAL                  544668 non-null  object 

## INVOICE_CURRENCY_CODE: filter USD rows / drop column

In [13]:
data = data[data['INVOICE_CURRENCY_CODE'] == "USD"]

In [14]:
data.shape

(544606, 64)

In [15]:
data.drop(['INVOICE_CURRENCY_CODE'], axis = 1, inplace = True)

In [16]:
data.shape

(544606, 63)

## SALES_CHANNEL: drop NA

In [17]:
# drop rows with null values in SALES_CHANNEL
data = data.dropna(axis = 0, subset = ['SALES_CHANNEL'])

In [18]:
data.shape

(544603, 63)

## QUANTITY: convert to nearest integer

In [19]:
data['QUANTITY'] = data['QUANTITY'].apply(np.ceil).astype(int)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544603 entries, 0 to 628076
Data columns (total 63 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544603 non-null  int64         
 1   SLS_USD                         544603 non-null  float64       
 2   Qty                             544603 non-null  float64       
 3   Site_Level_Price_Index_WS       544603 non-null  float64       
 4   Site_Level_Price_Index_STU      544603 non-null  float64       
 5   Site_Level_Price_Index          544603 non-null  float64       
 6   CUSTOMER_ID                     544603 non-null  int64         
 7   CUSTOMER_SITE_ID                544603 non-null  int64         
 8   SITE_CREATION_DATE              544603 non-null  datetime64[ns]
 9   SIC_CODE_DESC                   544603 non-null  object        
 10  SHORT_VERTICAL                  544603 non-null  object 

## SHIP_TO_SITE_ID / SHIP_TO_SITE_NUMBER / SHIP_TO_CUSTOMER_ID / BILL_TO_CUSTOMER_ID: drop column

In [21]:
data.drop(["SHIP_TO_SITE_ID","SHIP_TO_SITE_NUMBER", "SHIP_TO_CUSTOMER_ID", "BILL_TO_CUSTOMER_ID"],axis = 1, inplace = True)

In [22]:
data.shape

(544603, 59)

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544603 entries, 0 to 628076
Data columns (total 59 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544603 non-null  int64         
 1   SLS_USD                         544603 non-null  float64       
 2   Qty                             544603 non-null  float64       
 3   Site_Level_Price_Index_WS       544603 non-null  float64       
 4   Site_Level_Price_Index_STU      544603 non-null  float64       
 5   Site_Level_Price_Index          544603 non-null  float64       
 6   CUSTOMER_ID                     544603 non-null  int64         
 7   CUSTOMER_SITE_ID                544603 non-null  int64         
 8   SITE_CREATION_DATE              544603 non-null  datetime64[ns]
 9   SIC_CODE_DESC                   544603 non-null  object        
 10  SHORT_VERTICAL                  544603 non-null  object 

## PRODUCT_GROUP: drop column

In [24]:
data.drop(["PRODUCT_GROUP"], axis = 1, inplace = True)

In [25]:
data.shape

(544603, 58)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544603 entries, 0 to 628076
Data columns (total 58 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544603 non-null  int64         
 1   SLS_USD                         544603 non-null  float64       
 2   Qty                             544603 non-null  float64       
 3   Site_Level_Price_Index_WS       544603 non-null  float64       
 4   Site_Level_Price_Index_STU      544603 non-null  float64       
 5   Site_Level_Price_Index          544603 non-null  float64       
 6   CUSTOMER_ID                     544603 non-null  int64         
 7   CUSTOMER_SITE_ID                544603 non-null  int64         
 8   SITE_CREATION_DATE              544603 non-null  datetime64[ns]
 9   SIC_CODE_DESC                   544603 non-null  object        
 10  SHORT_VERTICAL                  544603 non-null  object 

## SIC_CODE_DESC: drop column

In [27]:
data.drop(["SIC_CODE_DESC"],axis = 1, inplace = True)

In [28]:
data.shape

(544603, 57)

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544603 entries, 0 to 628076
Data columns (total 57 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544603 non-null  int64         
 1   SLS_USD                         544603 non-null  float64       
 2   Qty                             544603 non-null  float64       
 3   Site_Level_Price_Index_WS       544603 non-null  float64       
 4   Site_Level_Price_Index_STU      544603 non-null  float64       
 5   Site_Level_Price_Index          544603 non-null  float64       
 6   CUSTOMER_ID                     544603 non-null  int64         
 7   CUSTOMER_SITE_ID                544603 non-null  int64         
 8   SITE_CREATION_DATE              544603 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  544603 non-null  object        
 10  CITY                            544603 non-null  object 

## COUNTRY: drop column

In [30]:
data.drop(["COUNTRY"],axis = 1, inplace = True)

In [31]:
data.shape

(544603, 56)

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544603 entries, 0 to 628076
Data columns (total 56 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544603 non-null  int64         
 1   SLS_USD                         544603 non-null  float64       
 2   Qty                             544603 non-null  float64       
 3   Site_Level_Price_Index_WS       544603 non-null  float64       
 4   Site_Level_Price_Index_STU      544603 non-null  float64       
 5   Site_Level_Price_Index          544603 non-null  float64       
 6   CUSTOMER_ID                     544603 non-null  int64         
 7   CUSTOMER_SITE_ID                544603 non-null  int64         
 8   SITE_CREATION_DATE              544603 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  544603 non-null  object        
 10  CITY                            544603 non-null  object 

## CITY: convert  to upper

In [33]:
data['CITY'] = data['CITY'].str.upper()

## POSTAL_CODE: remove NA and "SHIP TO" rows and retain first 5 digits

In [34]:
data = data[data['POSTAL_CODE'] != "SHIP TO"]

In [35]:
data = data.dropna(axis=0, subset=['POSTAL_CODE'])

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544594 entries, 0 to 628076
Data columns (total 56 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544594 non-null  int64         
 1   SLS_USD                         544594 non-null  float64       
 2   Qty                             544594 non-null  float64       
 3   Site_Level_Price_Index_WS       544594 non-null  float64       
 4   Site_Level_Price_Index_STU      544594 non-null  float64       
 5   Site_Level_Price_Index          544594 non-null  float64       
 6   CUSTOMER_ID                     544594 non-null  int64         
 7   CUSTOMER_SITE_ID                544594 non-null  int64         
 8   SITE_CREATION_DATE              544594 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  544594 non-null  object        
 10  CITY                            544594 non-null  object 

In [37]:
data['POSTAL_CODE'] = data['POSTAL_CODE'].str[:5]

## SELLING_ORG: drop column

In [38]:
data.drop(['SELLING_ORG'],axis = 1, inplace = True)

In [39]:
data.shape

(544594, 55)

## CORPORATE_CODE: convert to STRATEGIC_ACCOUNTS

In [40]:
# identifying strategic accounts
prefix = ['PA', 'SA', 'TA', 'IA', 'NA']
data['STRATEGIC_ACCOUNTS'] = data.CORPORATE_CODE.str.startswith(tuple(prefix), na = False).astype(int) 
data.drop(columns = 'CORPORATE_CODE', inplace = True)

In [41]:
data.head()

,ITEM_ID,SLS_USD,Qty,Site_Level_Price_Index_WS,Site_Level_Price_Index_STU,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SITE_CREATION_DATE,SHORT_VERTICAL,CITY,STATE,POSTAL_CODE,CUSTOMER_CLASS,LAST_MODIFIED_DATE,TERRITORY_REGION,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,IB_CONFIRMED_DATE,IB_STATUS,IB_STATUS_INACTIVE_REASON,DUNS_NUMBER,CUSTOMER_TRX_ID,DW_INVOICE_ID,TRX_DATE,TRX_AMT_USD,TRX_COST_USD,SALES_CHANNEL,INVOICE_TYPE,QUANTITY,INVOICE_NUM,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,UOM,PFV_FAMILY,PFV_MODEL_GROUP,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Tasks,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Install_Bases,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS
0,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,2013-08-15,FRUIT & VEGETABLE,SALEM,OR,97301,END USER,2019-09-30,NW,Industrial,S,Over Stocked / Timing,2017-04-10,Active,NaN,78842640,8731929,108745302,2016-02-05,207.72,19.13,Online,INVOICE,1,3206875,STANDARD DOMESTIC,"511,759,211.00",CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,nan,NaN,NaN,6.00,0.00,0.00,6.00,No Contract,0.00,0
1,598740,644.37,3.00,588.99,644.37,0.91,113032,578406,2013-03-25,AERO/AUTO,CRESTVIEW,FL,32539,END USER,2019-10-01,SE,Industrial,S,NaN,2020-05-08,Active,NaN,43202248,9305874,276000052,2016-12-16,214.79,15.85,Esker,INVOICE,1,3294491,STANDARD DOMESTIC,"511,868,043.00",CIJ,VALUE PACK,Each,CIJ,CIJ - LEGACY,21.00,5.00,16.00,Email,58.00,62.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",4.00,FSMA,0.00,0
2,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,2013-08-15,FRUIT & VEGETABLE,SALEM,OR,97301,END USER,2019-09-30,NW,Industrial,S,Over Stocked / Timing,2017-04-10,Active,NaN,78842640,9093354,223068431,2016-08-26,207.72,19.13,Online,INVOICE,1,3263477,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,nan,NaN,NaN,6.00,0.00,0.00,6.00,No Contract,0.00,0
3,598740,644.37,3.00,588.99,644.37,0.91,113032,578406,2013-03-25,AERO/AUTO,CRESTVIEW,FL,32539,END USER,2019-10-01,SE,Industrial,S,NaN,2020-05-08,Active,NaN,43202248,9195970,247802392,2016-10-19,429.58,31.69,Esker,INVOICE,2,3278903,STANDARD DOMESTIC,"511,849,315.00",CIJ,VALUE PACK,Each,CIJ,CIJ - LEGACY,21.00,5.00,16.00,Email,58.00,62.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",4.00,FSMA,0.00,0
4,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,2013-08-15,FRUIT & VEGETABLE,SALEM,OR,97301,END USER,2019-09-30,NW,Industrial,S,Over Stocked / Timing,2017-04-10,Active,NaN,78842640,9079815,219738849,2016-08-19,623.16,57.39,Online,INVOICE,3,3261624,STANDARD DOMESTIC,"511,823,154.00",CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,nan,NaN,NaN,6.00,0.00,0.00,6.00,No Contract,0.00,0


In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544594 entries, 0 to 628076
Data columns (total 55 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544594 non-null  int64         
 1   SLS_USD                         544594 non-null  float64       
 2   Qty                             544594 non-null  float64       
 3   Site_Level_Price_Index_WS       544594 non-null  float64       
 4   Site_Level_Price_Index_STU      544594 non-null  float64       
 5   Site_Level_Price_Index          544594 non-null  float64       
 6   CUSTOMER_ID                     544594 non-null  int64         
 7   CUSTOMER_SITE_ID                544594 non-null  int64         
 8   SITE_CREATION_DATE              544594 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  544594 non-null  object        
 10  CITY                            544594 non-null  object 

## TERRITORY_REGION: reassign with mode of each state



In [43]:
# Removing Graphics, OEM, Distributor, Postal, Canada, NAs
# keeping MC, NW, NE, MW, SE, SC
data['TERRITORY_REGION'].replace(['Graphics','OEM','Distributor','Postal','Canada'], '-', inplace = True)
data['TERRITORY_REGION'].fillna('-', inplace = True)
data['TERRITORY_REGION'] = data['TERRITORY_REGION'].replace('-', np.nan)

In [44]:
from scipy import stats
region = data.groupby(['STATE'])['TERRITORY_REGION'].agg(lambda x: stats.mode(x)[0][0]).reset_index()
region['TERRITORY_REGION'][region['STATE'] == 'DC'] = 'NE'

In [45]:
data.drop(columns = 'TERRITORY_REGION', inplace = True)

In [46]:
# reassigning values
data = pd.merge(data, region, how = 'left', on = 'STATE')

In [47]:
data.TERRITORY_REGION.value_counts()

NE    109893
MC     95730
NW     95154
MW     90512
SE     86455
SC     66850
Name: TERRITORY_REGION, dtype: int64

In [48]:
data.shape

(544594, 55)

## TERRITORY_TYPE: replace NA with mode of each city or state

In [49]:
data['TERRITORY_TYPE'].value_counts()

Industrial    516671
Graphics       25345
Postal          1451
Name: TERRITORY_TYPE, dtype: int64

In [50]:
# filling NAs using mode by CITY if mode exists
data['TERRITORY_TYPE'] = data['TERRITORY_TYPE'].fillna(data.groupby('CITY')['TERRITORY_TYPE'].transform(lambda x: stats.mode(x)[0][0])).astype(str)

In [51]:
# filling other using mode by STATE
data['TERRITORY_TYPE'] = data['TERRITORY_TYPE'].replace('0', np.NaN)
data['TERRITORY_TYPE'] = data['TERRITORY_TYPE'].fillna(data.groupby('STATE')['TERRITORY_TYPE'].transform(lambda x: stats.mode(x)[0][0])).astype(str)

In [52]:
data['TERRITORY_TYPE'].value_counts()

Industrial    517778
Graphics       25365
Postal          1451
Name: TERRITORY_TYPE, dtype: int64

In [53]:
data.shape

(544594, 55)

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544594 entries, 0 to 544593
Data columns (total 55 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         544594 non-null  int64         
 1   SLS_USD                         544594 non-null  float64       
 2   Qty                             544594 non-null  float64       
 3   Site_Level_Price_Index_WS       544594 non-null  float64       
 4   Site_Level_Price_Index_STU      544594 non-null  float64       
 5   Site_Level_Price_Index          544594 non-null  float64       
 6   CUSTOMER_ID                     544594 non-null  int64         
 7   CUSTOMER_SITE_ID                544594 non-null  int64         
 8   SITE_CREATION_DATE              544594 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  544594 non-null  object        
 10  CITY                            544594 non-null  object 

## SUPPLIES_DECLINE_REASONS: replace NA with "None" and combine some values

In [55]:
data["SUPPLIES_DECLINE_REASONS"].value_counts()

Migration to 1000 Line                    36913
Duplicate Site                            26939
AP Competitive Displacement               26598
Over Stocked                              24964
Production / Code Reduction               22956
Over Stocked / Timing                     21730
Data Integration                          19410
Off Brand                                 13226
Production Down (timing)                  10720
Migration to 1000 Line/TIJ/TTO/LCM/LPA     7921
Site Closed                                7467
Migration to Lasers                        4437
Served by Authorized Distributor           3755
Code Reduction                             3557
Recent Regain/Win-back                     3444
Recent regain / win back                   2470
Seasonal Producer                          2237
Partially Moved Equipment                  1490
Moved Equipment                            1221
Project Based                              1194
Seasonal                                

In [56]:
data["SUPPLIES_DECLINE_REASONS"] = data["SUPPLIES_DECLINE_REASONS"].fillna('None')

In [57]:
data['SUPPLIES_DECLINE_REASONS']=data['SUPPLIES_DECLINE_REASONS'].\
    replace('Migration to 1000 Line','Migration to 1000 Line/TIJ/TTO/LCM/LPA')
data['SUPPLIES_DECLINE_REASONS']=data['SUPPLIES_DECLINE_REASONS'].\
    replace('Over Stocked','Over Stocked / Timing')
data['SUPPLIES_DECLINE_REASONS']=data['SUPPLIES_DECLINE_REASONS'].\
    replace('Recent regain / win back','Recent Regain/Win-back')
data['SUPPLIES_DECLINE_REASONS']=data['SUPPLIES_DECLINE_REASONS'].\
    replace('Credit Hold','Financial Distress/Credit Hold')
data['SUPPLIES_DECLINE_REASONS']=data['SUPPLIES_DECLINE_REASONS'].\
    replace('Code Reduction','Production / Code Reduction')
data['SUPPLIES_DECLINE_REASONS']=data['SUPPLIES_DECLINE_REASONS'].\
    replace('Seasonal','Seasonal Producer')
data['SUPPLIES_DECLINE_REASONS']=data['SUPPLIES_DECLINE_REASONS'].\
    replace('Partially Moved Equipment','Moved Equipment')

In [58]:
data["SUPPLIES_DECLINE_REASONS"].value_counts()

None                                      299096
Over Stocked / Timing                      46694
Migration to 1000 Line/TIJ/TTO/LCM/LPA     44834
Duplicate Site                             26939
AP Competitive Displacement                26598
Production / Code Reduction                26513
Data Integration                           19410
Off Brand                                  13226
Production Down (timing)                   10720
Site Closed                                 7467
Recent Regain/Win-back                      5914
Migration to Lasers                         4437
Served by Authorized Distributor            3755
Seasonal Producer                           3336
Moved Equipment                             2711
Project Based                               1194
No More Coding Requirement                  1040
Printing/EQ downtime Issues                  353
VJ Operations Issues                         182
Financial Distress/Credit Hold               117
Pricing / Discountin

In [59]:
# Duplicates and Data Integration refers to shift of business to other sites and hence removed to avoid skewness
data = data[(data['SUPPLIES_DECLINE_REASONS'] != "Data Integration") & (data['SUPPLIES_DECLINE_REASONS'] != "Duplicate Site")]

## IB_CONFIRMED_DATE: drop column

In [60]:
data.drop(['IB_CONFIRMED_DATE'],axis = 1, inplace = True)

In [61]:
data.shape

(498245, 54)

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498245 entries, 0 to 544593
Data columns (total 54 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         498245 non-null  int64         
 1   SLS_USD                         498245 non-null  float64       
 2   Qty                             498245 non-null  float64       
 3   Site_Level_Price_Index_WS       498245 non-null  float64       
 4   Site_Level_Price_Index_STU      498245 non-null  float64       
 5   Site_Level_Price_Index          498245 non-null  float64       
 6   CUSTOMER_ID                     498245 non-null  int64         
 7   CUSTOMER_SITE_ID                498245 non-null  int64         
 8   SITE_CREATION_DATE              498245 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  498245 non-null  object        
 10  CITY                            498245 non-null  object 

## IB_STATUS: replace NA with "Active" as last modified date is after 2017 

In [63]:
data['IB_STATUS'].value_counts()

Active      469693
Inactive     20178
Name: IB_STATUS, dtype: int64

In [64]:
data['IB_STATUS'].fillna("Active", inplace = True)

In [65]:
data.shape

(498245, 54)

In [66]:
data['IB_STATUS'].value_counts()

Active      478067
Inactive     20178
Name: IB_STATUS, dtype: int64

In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498245 entries, 0 to 544593
Data columns (total 54 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         498245 non-null  int64         
 1   SLS_USD                         498245 non-null  float64       
 2   Qty                             498245 non-null  float64       
 3   Site_Level_Price_Index_WS       498245 non-null  float64       
 4   Site_Level_Price_Index_STU      498245 non-null  float64       
 5   Site_Level_Price_Index          498245 non-null  float64       
 6   CUSTOMER_ID                     498245 non-null  int64         
 7   CUSTOMER_SITE_ID                498245 non-null  int64         
 8   SITE_CREATION_DATE              498245 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  498245 non-null  object        
 10  CITY                            498245 non-null  object 

## IB_STATUS_INACTIVE_REASON: drop column

In [68]:
data["IB_STATUS_INACTIVE_REASON"].value_counts()

Site Closed                    4471
AP competitive displacement    3414
Moved equipment                 863
No more coding requirement      806
Data Integration                607
Duplicate Site                   73
None                             39
Name: IB_STATUS_INACTIVE_REASON, dtype: int64

In [69]:
data.drop(["IB_STATUS_INACTIVE_REASON"],axis = 1, inplace = True)

In [70]:
data.shape

(498245, 53)

In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498245 entries, 0 to 544593
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         498245 non-null  int64         
 1   SLS_USD                         498245 non-null  float64       
 2   Qty                             498245 non-null  float64       
 3   Site_Level_Price_Index_WS       498245 non-null  float64       
 4   Site_Level_Price_Index_STU      498245 non-null  float64       
 5   Site_Level_Price_Index          498245 non-null  float64       
 6   CUSTOMER_ID                     498245 non-null  int64         
 7   CUSTOMER_SITE_ID                498245 non-null  int64         
 8   SITE_CREATION_DATE              498245 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  498245 non-null  object        
 10  CITY                            498245 non-null  object 

## Site_Level_Price_Index: replace NA by grouping

In [72]:
len(data[data['Site_Level_Price_Index'] == 0])

1461

In [73]:
data['Site_Level_Price_Index'].replace('0', np.nan, inplace = True)

In [74]:
data['TRX_YEAR'] = pd.DatetimeIndex(data['TRX_DATE']).year

In [75]:
data['Site_Level_Price_Index'] = data.groupby(['CUSTOMER_SITE_ID', 'ITEM_ID', 'TRX_YEAR'])['Site_Level_Price_Index'].transform(lambda x: x.fillna(x.mean()))
data['Site_Level_Price_Index'] = data.groupby(['CUSTOMER_ID', 'ITEM_ID', 'TRX_YEAR'])['Site_Level_Price_Index'].transform(lambda x: x.fillna(x.mean()))
data['Site_Level_Price_Index'] = data.groupby(['ITEM_ID', 'TRX_YEAR'])['Site_Level_Price_Index'].transform(lambda x: x.fillna(x.mean()))

In [76]:
data['Site_Level_Price_Index'] = data.groupby(['CUSTOMER_SITE_ID', 'TRX_YEAR'])['Site_Level_Price_Index'].transform(lambda x: x.fillna(x.mean()))
data['Site_Level_Price_Index'] = data.groupby(['CUSTOMER_ID', 'TRX_YEAR'])['Site_Level_Price_Index'].transform(lambda x: x.fillna(x.mean()))

In [77]:
data['Site_Level_Price_Index'] = data.groupby(['TRX_YEAR'])['Site_Level_Price_Index'].transform(lambda x: x.fillna(x.mean()))

In [78]:
data['Site_Level_Price_Index'].isna().sum()

0

In [79]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498245 entries, 0 to 544593
Data columns (total 54 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         498245 non-null  int64         
 1   SLS_USD                         498245 non-null  float64       
 2   Qty                             498245 non-null  float64       
 3   Site_Level_Price_Index_WS       498245 non-null  float64       
 4   Site_Level_Price_Index_STU      498245 non-null  float64       
 5   Site_Level_Price_Index          498245 non-null  float64       
 6   CUSTOMER_ID                     498245 non-null  int64         
 7   CUSTOMER_SITE_ID                498245 non-null  int64         
 8   SITE_CREATION_DATE              498245 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  498245 non-null  object        
 10  CITY                            498245 non-null  object 

## Total_SVC_Incidents / Total_Repeat_Calls / Total_FTF_Calls: replace NA by grouping

In [80]:
# filling NAs using mean of customer, customer class
data['Total_SVC_Incidents'] = data.groupby('CUSTOMER_SITE_ID')['Total_SVC_Incidents'].transform(lambda x: x.fillna(x.mean()))
data['Total_SVC_Incidents'] = data.groupby('CUSTOMER_ID')['Total_SVC_Incidents'].transform(lambda x: x.fillna(x.mean()))
data['Total_SVC_Incidents'] = data.groupby('CUSTOMER_CLASS')['Total_SVC_Incidents'].transform(lambda x: x.fillna(x.mean()))

In [81]:
data['Total_SVC_Incidents'].isna().sum()

0

In [82]:
# filling NAs using mean of customer, customer class
data['Total_Repeat_Calls'] = data.groupby('CUSTOMER_SITE_ID')['Total_Repeat_Calls'].transform(lambda x: x.fillna(x.mean()))
data['Total_Repeat_Calls'] = data.groupby('CUSTOMER_ID')['Total_Repeat_Calls'].transform(lambda x: x.fillna(x.mean()))
data['Total_Repeat_Calls'] = data.groupby('CUSTOMER_CLASS')['Total_Repeat_Calls'].transform(lambda x: x.fillna(x.mean()))

In [83]:
data['Total_Repeat_Calls'].isna().sum()

0

In [84]:
data['Total_FTF_Calls'] = data.groupby('CUSTOMER_SITE_ID')['Total_FTF_Calls'].transform(lambda x: x.fillna(x.mean()))
data['Total_FTF_Calls'] = data.groupby('CUSTOMER_ID')['Total_FTF_Calls'].transform(lambda x: x.fillna(x.mean()))
data['Total_FTF_Calls'] = data.groupby('CUSTOMER_CLASS')['Total_FTF_Calls'].transform(lambda x: x.fillna(x.mean()))

In [85]:
data['Total_FTF_Calls'].isna().sum()

0

## Most_Frequent_Interaction_Type: replace NA using "Call"

In [86]:
data['Most_Frequent_Interaction_Type'].value_counts()

Call                         413396
Email                         42770
Other                         31697
Meeting                        3030
Customer Meeting                747
Dial                             70
Contact Customer                 49
Make Qualified Sales Call        23
Callback                         15
TS Task                          13
Survey                            1
Name: Most_Frequent_Interaction_Type, dtype: int64

In [87]:
data['Most_Frequent_Interaction_Type'] = data['Most_Frequent_Interaction_Type'].replace(np.nan,"Call")

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498245 entries, 0 to 544593
Data columns (total 54 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         498245 non-null  int64         
 1   SLS_USD                         498245 non-null  float64       
 2   Qty                             498245 non-null  float64       
 3   Site_Level_Price_Index_WS       498245 non-null  float64       
 4   Site_Level_Price_Index_STU      498245 non-null  float64       
 5   Site_Level_Price_Index          498245 non-null  float64       
 6   CUSTOMER_ID                     498245 non-null  int64         
 7   CUSTOMER_SITE_ID                498245 non-null  int64         
 8   SITE_CREATION_DATE              498245 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  498245 non-null  object        
 10  CITY                            498245 non-null  object 

## Total_Visits / Total_Tasks: replace NA by grouping

In [89]:
# filling NAs using mean of customer, customer class
data['Total_Visits'] = data.groupby('CUSTOMER_SITE_ID')['Total_Visits'].transform(lambda x: x.fillna(x.mean()))
data['Total_Visits'] = data.groupby('CUSTOMER_ID')['Total_Visits'].transform(lambda x: x.fillna(x.mean()))
data['Total_Visits'] = data.groupby('CUSTOMER_CLASS')['Total_Visits'].transform(lambda x: x.fillna(x.mean()))

data['Total_Tasks'] = data.groupby('CUSTOMER_SITE_ID')['Total_Tasks'].transform(lambda x: x.fillna(x.mean()))
data['Total_Tasks'] = data.groupby('CUSTOMER_ID')['Total_Tasks'].transform(lambda x: x.fillna(x.mean()))
data['Total_Tasks'] = data.groupby('CUSTOMER_CLASS')['Total_Tasks'].transform(lambda x: x.fillna(x.mean()))

In [90]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498245 entries, 0 to 544593
Data columns (total 54 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         498245 non-null  int64         
 1   SLS_USD                         498245 non-null  float64       
 2   Qty                             498245 non-null  float64       
 3   Site_Level_Price_Index_WS       498245 non-null  float64       
 4   Site_Level_Price_Index_STU      498245 non-null  float64       
 5   Site_Level_Price_Index          498245 non-null  float64       
 6   CUSTOMER_ID                     498245 non-null  int64         
 7   CUSTOMER_SITE_ID                498245 non-null  int64         
 8   SITE_CREATION_DATE              498245 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  498245 non-null  object        
 10  CITY                            498245 non-null  object 

## Total_Cases: replace NA by grouping

In [91]:
# filling NAs using mean of customer, customer class
data['Total_Cases'] = data.groupby('CUSTOMER_SITE_ID')['Total_Cases'].transform(lambda x: x.fillna(x.mean()))
data['Total_Cases'] = data.groupby('CUSTOMER_ID')['Total_Cases'].transform(lambda x: x.fillna(x.mean()))
data['Total_Cases'] = data.groupby('CUSTOMER_CLASS')['Total_Cases'].transform(lambda x: x.fillna(x.mean()))

In [92]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498245 entries, 0 to 544593
Data columns (total 54 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         498245 non-null  int64         
 1   SLS_USD                         498245 non-null  float64       
 2   Qty                             498245 non-null  float64       
 3   Site_Level_Price_Index_WS       498245 non-null  float64       
 4   Site_Level_Price_Index_STU      498245 non-null  float64       
 5   Site_Level_Price_Index          498245 non-null  float64       
 6   CUSTOMER_ID                     498245 non-null  int64         
 7   CUSTOMER_SITE_ID                498245 non-null  int64         
 8   SITE_CREATION_DATE              498245 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  498245 non-null  object        
 10  CITY                            498245 non-null  object 

## Max_Case_Origin/Max_Case_Reason: replace NA with 'unknown'

In [93]:
data['Max_Case_Origin'].value_counts()

Email - VTI NACC                    94923
Phone                               50717
CC Survey Followup                  16512
TS Survey Followup                   6771
FS Survey Followup                   2634
Email/Fax - VTI CS                   1856
Install Complete                      631
CX Survey Detractor                   583
Email                                 273
Email - VTI CC Sales Escalations        3
Name: Max_Case_Origin, dtype: int64

In [94]:
data['Max_Case_Reason'].value_counts()

Customer Experience    146775
CX: Customer Care       13882
CX: Tech Support         7849
CX: Field Service        4279
CX: Credit               1351
CX: Field Sales           363
CX: Other Team            349
CX: Manufacturing          55
Name: Max_Case_Reason, dtype: int64

In [95]:
data['Max_Case_Origin'] = data['Max_Case_Origin'].fillna('unknown')
data['Max_Case_Reason'] = data['Max_Case_Reason'].fillna('unknown')

In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498245 entries, 0 to 544593
Data columns (total 54 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   ITEM_ID                         498245 non-null  int64         
 1   SLS_USD                         498245 non-null  float64       
 2   Qty                             498245 non-null  float64       
 3   Site_Level_Price_Index_WS       498245 non-null  float64       
 4   Site_Level_Price_Index_STU      498245 non-null  float64       
 5   Site_Level_Price_Index          498245 non-null  float64       
 6   CUSTOMER_ID                     498245 non-null  int64         
 7   CUSTOMER_SITE_ID                498245 non-null  int64         
 8   SITE_CREATION_DATE              498245 non-null  datetime64[ns]
 9   SHORT_VERTICAL                  498245 non-null  object        
 10  CITY                            498245 non-null  object 

## Contract_Category: drop rows with NA

In [97]:
data = data.dropna(axis=0, subset=['Contract_Category'])

# Converting data types

In [98]:
# convert to right dtype
data['CUSTOMER_SITE_ID'] = data['CUSTOMER_SITE_ID'].astype(str)
data['CUSTOMER_ID'] = data['CUSTOMER_ID'].astype(str)
data['CUSTOMER_TRX_ID'] = data['CUSTOMER_TRX_ID'].astype(str)
data['DW_INVOICE_ID'] = data['DW_INVOICE_ID'].astype(str)
data['ITEM_ID'] = data['ITEM_ID'].astype(str)
data['INVOICE_NUM'] = data['INVOICE_NUM'].astype(str)
data['ORDER_NUM'] = data['ORDER_NUM'].astype(str)

In [99]:
data.head()

,ITEM_ID,SLS_USD,Qty,Site_Level_Price_Index_WS,Site_Level_Price_Index_STU,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SITE_CREATION_DATE,SHORT_VERTICAL,CITY,STATE,POSTAL_CODE,CUSTOMER_CLASS,LAST_MODIFIED_DATE,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,IB_STATUS,DUNS_NUMBER,CUSTOMER_TRX_ID,DW_INVOICE_ID,TRX_DATE,TRX_AMT_USD,TRX_COST_USD,SALES_CHANNEL,INVOICE_TYPE,QUANTITY,INVOICE_NUM,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,UOM,PFV_FAMILY,PFV_MODEL_GROUP,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Tasks,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Install_Bases,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR
0,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,2013-08-15,FRUIT & VEGETABLE,SALEM,OR,97301,END USER,2019-09-30,Industrial,S,Over Stocked / Timing,Active,78842640,8731929,108745302,2016-02-05,207.72,19.13,Online,INVOICE,1,3206875,STANDARD DOMESTIC,511759211.0,CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,1.97,unknown,unknown,6.00,0.00,0.00,6.00,No Contract,0.00,0,NW,2016
1,598740,644.37,3.00,588.99,644.37,0.91,113032,578406,2013-03-25,AERO/AUTO,CRESTVIEW,FL,32539,END USER,2019-10-01,Industrial,S,None,Active,43202248,9305874,276000052,2016-12-16,214.79,15.85,Esker,INVOICE,1,3294491,STANDARD DOMESTIC,511868043.0,CIJ,VALUE PACK,Each,CIJ,CIJ - LEGACY,21.00,5.00,16.00,Email,58.00,62.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",4.00,FSMA,0.00,0,SE,2016
2,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,2013-08-15,FRUIT & VEGETABLE,SALEM,OR,97301,END USER,2019-09-30,Industrial,S,Over Stocked / Timing,Active,78842640,9093354,223068431,2016-08-26,207.72,19.13,Online,INVOICE,1,3263477,STANDARD DOMESTIC,511823154.0,CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,1.97,unknown,unknown,6.00,0.00,0.00,6.00,No Contract,0.00,0,NW,2016
3,598740,644.37,3.00,588.99,644.37,0.91,113032,578406,2013-03-25,AERO/AUTO,CRESTVIEW,FL,32539,END USER,2019-10-01,Industrial,S,None,Active,43202248,9195970,247802392,2016-10-19,429.58,31.69,Esker,INVOICE,2,3278903,STANDARD DOMESTIC,511849315.0,CIJ,VALUE PACK,Each,CIJ,CIJ - LEGACY,21.00,5.00,16.00,Email,58.00,62.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",4.00,FSMA,0.00,0,SE,2016
4,268183,"1,246.32",6.00,"1,001.87","1,246.32",0.80,117841,609636,2013-08-15,FRUIT & VEGETABLE,SALEM,OR,97301,END USER,2019-09-30,Industrial,S,Over Stocked / Timing,Active,78842640,9079815,219738849,2016-08-19,623.16,57.39,Online,INVOICE,3,3261624,STANDARD DOMESTIC,511823154.0,CIJ,MAKE-UP,Each,CIJ,CIJ - LEGACY,11.00,3.00,8.00,Call,18.00,17.00,1.97,unknown,unknown,6.00,0.00,0.00,6.00,No Contract,0.00,0,NW,2016


# Dropping non-relevant columns

In [100]:
data.drop(['ITEM_ID','SLS_USD','Qty','Site_Level_Price_Index_WS',
 'Site_Level_Price_Index_STU','SITE_CREATION_DATE','CITY',
 'STATE','IB_STATUS','CUSTOMER_TRX_ID',
 'DW_INVOICE_ID','Num_of_Install_Bases','Total_Tasks','PFV_FAMILY','PFV_MODEL_GROUP','UOM','INVOICE_NUM','INVOICE_TYPE','LAST_MODIFIED_DATE'],axis = 1,inplace = True)

In [101]:
# create Margin column
data['TRX_COST_USD'] = data['TRX_AMT_USD'] - data['TRX_COST_USD']

In [102]:
data.rename(columns={'TRX_COST_USD':'Margin'},inplace = True) 

In [103]:
data.shape

(460306, 35)

In [104]:
data.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR
0,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-05,207.72,188.59,Online,1,STANDARD DOMESTIC,511759211.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016
1,0.91,113032,578406,AERO/AUTO,32539,END USER,Industrial,S,None,43202248,2016-12-16,214.79,198.94,Esker,1,STANDARD DOMESTIC,511868043.0,CIJ,VALUE PACK,21.00,5.00,16.00,Email,58.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",FSMA,0.00,0,SE,2016
2,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-26,207.72,188.59,Online,1,STANDARD DOMESTIC,511823154.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016
3,0.91,113032,578406,AERO/AUTO,32539,END USER,Industrial,S,None,43202248,2016-10-19,429.58,397.89,Esker,2,STANDARD DOMESTIC,511849315.0,CIJ,VALUE PACK,21.00,5.00,16.00,Email,58.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",FSMA,0.00,0,SE,2016
4,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-19,623.16,565.77,Online,3,STANDARD DOMESTIC,511823154.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016


In [105]:
data.drop_duplicates(inplace = True)

# Dropping discrepancies

In [106]:
# contract mismatch rows to be removed
data = data.loc[~((data['Contract_Category'] == "No Contract") & (data['Contract_length'] > 0)),:]

In [107]:
# drop negative margin cases
data = data[data['Margin'] > 0]

In [108]:
data.shape

(380952, 35)

In [109]:
data.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR
0,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-05,207.72,188.59,Online,1,STANDARD DOMESTIC,511759211.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016
1,0.91,113032,578406,AERO/AUTO,32539,END USER,Industrial,S,None,43202248,2016-12-16,214.79,198.94,Esker,1,STANDARD DOMESTIC,511868043.0,CIJ,VALUE PACK,21.00,5.00,16.00,Email,58.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",FSMA,0.00,0,SE,2016
2,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-26,207.72,188.59,Online,1,STANDARD DOMESTIC,511823154.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016
3,0.91,113032,578406,AERO/AUTO,32539,END USER,Industrial,S,None,43202248,2016-10-19,429.58,397.89,Esker,2,STANDARD DOMESTIC,511849315.0,CIJ,VALUE PACK,21.00,5.00,16.00,Email,58.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",FSMA,0.00,0,SE,2016
4,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-19,623.16,565.77,Online,3,STANDARD DOMESTIC,511823154.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016


# Removing one-time purchasers

In [110]:
num_trxn_overall = data.groupby('CUSTOMER_SITE_ID')['TRX_DATE'].count().reset_index()
num_trxn_overall.rename(columns = {'TRX_DATE': 'Num_of_Trxns'}, inplace = True)

In [111]:
# remove customers with a single transaction (one-time purchasers)
single_purchaser = num_trxn_overall[num_trxn_overall['Num_of_Trxns'] <= 1]

In [112]:
single_purchaser_list = list(single_purchaser['CUSTOMER_SITE_ID'])

In [113]:
len(num_trxn_overall)

8918

In [114]:
data = data[~data['CUSTOMER_SITE_ID'].isin(single_purchaser_list)]

In [115]:
data.shape

(380401, 35)

In [116]:
data.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR
0,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-02-05,207.72,188.59,Online,1,STANDARD DOMESTIC,511759211.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016
1,0.91,113032,578406,AERO/AUTO,32539,END USER,Industrial,S,None,43202248,2016-12-16,214.79,198.94,Esker,1,STANDARD DOMESTIC,511868043.0,CIJ,VALUE PACK,21.00,5.00,16.00,Email,58.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",FSMA,0.00,0,SE,2016
2,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-26,207.72,188.59,Online,1,STANDARD DOMESTIC,511823154.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016
3,0.91,113032,578406,AERO/AUTO,32539,END USER,Industrial,S,None,43202248,2016-10-19,429.58,397.89,Esker,2,STANDARD DOMESTIC,511849315.0,CIJ,VALUE PACK,21.00,5.00,16.00,Email,58.00,1.00,Email - VTI NACC,Customer Experience,4.00,4.00,"1,011.25",FSMA,0.00,0,SE,2016
4,0.80,117841,609636,FRUIT & VEGETABLE,97301,END USER,Industrial,S,Over Stocked / Timing,78842640,2016-08-19,623.16,565.77,Online,3,STANDARD DOMESTIC,511823154.0,CIJ,MAKE-UP,11.00,3.00,8.00,Call,18.00,1.97,unknown,unknown,6.00,0.00,0.00,No Contract,0.00,0,NW,2016


# Exporting result table

In [117]:
data.to_csv("cleaned_data.csv")